In [1]:
!pip install --user gurobipy
import sys
print(sys.executable)





/opt/anaconda3/bin/python


In [2]:
import pandas as pd
from gurobipy import Model, GRB
import math
import copy
import time
from collections import defaultdict

start_time=time.time()

# Load the data from the Excel file
excel_file = "Scheduling Project Pilot.xlsx"
df = pd.read_excel(excel_file, sheet_name='Assignments')
df = df.rename(columns={df.columns[0]: 'Course'})
df = df.rename(columns={df.columns[1]: 'Instructor'})
df = df.rename(columns={df.columns[2]: 'Capacity'})
df = df.rename(columns={df.columns[3]: '# Sections'})
df = df.iloc[:, :-4]
print("df1=",df)
print(df["# Sections"].sum())
# Further filter the DataFrame to include only courses after "CS113" and exclude "CS115"
df = df.dropna(subset=['Course'])
df['Course_Number'] = df['Course'].str.extract(r'(\d+)')
df.rename(columns={'Section Size': 'Capacity'}, inplace=True)

df1=     Course          Instructor  Capacity  # Sections   70  Newark
0    CS114  Zaidenberg, Ayelet        80           2  160  Newark
1    CS114   Kapleau, Jonathan        80           1   80  Newark
2    CS116             Wu, Jun        35           2   70  Newark
3    CS241  Zaidenberg, Ayelet        80           1   80  Newark
4    CS241     Ionescu, Adrian        40           1   40  Newark
..     ...                 ...       ...         ...  ...     ...
125  IT340    Giannoglou, Karl        30           1   30  Newark
126  IT342    Giannoglou, Karl        30           1   30  Newark
127  IT420    Grayson, Tanisha        30           1   30  Newark
128  IT430     Senesy, Stanley        30           1   30  Newark
129  IT490       Patel, Dipesh        36           2   36  Newark

[130 rows x 6 columns]
160


In [3]:
def create_section_capacity_map(df):
    """
    Creates a dictionary mapping (course, instructor, section_number) to capacity.

    Args:
        df (DataFrame): DataFrame containing course, instructor, and capacity information.

    Returns:
        dict: A dictionary where keys are (course, instructor, section_number)
              and values are the capacity of the sections.
    """
    # Sort by Course, Instructor, and Capacity to ensure correct section numbering
    df_sorted = df.sort_values(by=['Course', 'Instructor', 'Capacity'])

    section_capacity_map = {}

    # Group by (course, instructor) and assign section numbers
    for (course, instructor), group in df_sorted.groupby(['Course', 'Instructor']):
        section_number = 1
        for _, row in group.iterrows():
            num_sections = int(row['# Sections'])
            capacity = row['Capacity']
            course = course.strip()
            instructor = instructor.strip()

            # Assign section numbers based on the sorted order
            for _ in range(num_sections):
                section_capacity_map[(course, instructor, section_number)] = capacity
                section_number += 1

    return section_capacity_map






def create_aggregated_dataframe(df):
    # Group by (Course, Instructor) and sum the number of sections
    aggregated_df = df.groupby(['Course', 'Instructor', 'Course_Number','Email']).agg(
        {'# Sections': 'sum'}
    ).reset_index()

    return aggregated_df

df_pre_scheduled = pd.read_excel(excel_file, sheet_name='pre-scheduled')
new_df = pd.read_excel(excel_file,sheet_name='Faculty')
new_df = new_df.rename(columns={new_df.columns[0]: 'Instructor'})
print("new_df=",new_df)

# Assuming the new_df also has an 'Instructor' column with professor names,
# Perform a merge on the 'Instructor' column
df = pd.merge(df, new_df, on='Instructor', how='left')
print(df)

new_df=             Instructor              Email  8 Digit ID  \
0    Abduallah, Yasser      ya54@njit.edu    31234231   
1        Amin, Nadyrah     nma46@njit.edu    21280977   
2    Basu Roy, Senjuti  senjutib@njit.edu    31405694   
3    Apostolyuk, Vadym      va58@njit.edu    31005195   
4       Arafeh, Bassel      ba62@njit.edu    31544114   
..                 ...                ...         ...   
178      Zhang, Genwei       gz6@njit.edu    31521469   
179       Yuan, Chenxi     cy324@njit.edu    31702048   
180         Zhang, Lei     lz392@njit.edu    31702050   
181       Zhang, Yijie     yz829@njit.edu    31481772   
182   Zunnurhain, Kazi  kzunnurh@njit.edu    31558650   

                       Job Title  
0                Adjunct, Tier I  
1                Adjunct, Tier I  
2           Professor, Associate  
3              Adjunct, Tier III  
4    University Lecturer, Senior  
..                           ...  
178                  PhD student  
179         Professor, Assi

In [4]:
section_capacity_map = create_section_capacity_map(df)

# Print the section capacity map
#print("Section Capacity Map:")
#for key, value in section_capacity_map.items():
#    print(f"{key}: Capacity {value}")

# Step 2: Create the aggregated DataFrame with total sections for each (Course, Instructor)
aggregated_df = create_aggregated_dataframe(df)

# Print the new DataFrame
print("\nAggregated DataFrame:")
print(aggregated_df)
df=copy.deepcopy(aggregated_df)


# Drop the helper column used for filtering
#df = df.drop(columns=['Course_Number'])

# Define the time slots and days (including Saturday, but minimizing its usage)
time_slots = [
    "8:30-10:00 AM",
    "10:00-11:30 AM",
    "11:30-1:00 PM",
    "1:00-2:30 PM",
    "2:30-4:00 PM",
    "4:00-5:30 PM",
    "6:00-7:30 PM",
    "7:30-9:00 PM"
]

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

# Load the balance percentages from the table in the image
slot_percentages = {
    ("Monday", "8:30-10:00 AM"): 0.20,
    ("Tuesday", "8:30-10:00 AM"): 0.20,
    ("Wednesday", "8:30-10:00 AM"): 0.25,
    ("Thursday", "8:30-10:00 AM"): 0.20,
    ("Friday", "8:30-10:00 AM"): 0.20,
    
    
    ("Monday", "10:00-11:30 AM"): 0.20,
    ("Tuesday", "10:00-11:30 AM"): 0.20,
    ("Wednesday", "10:00-11:30 AM"): 0.25,
    ("Thursday", "10:00-11:30 AM"): 0.20,
    ("Friday", "10:00-11:30 AM"): 0.20,
    
    
    ("Monday", "11:30-1:00 PM"): 0.20,
    ("Tuesday", "11:30-1:00 PM"): 0.20,
    ("Wednesday", "11:30-1:00 PM"): 0.25,
    ("Thursday", "11:30-1:00 PM"): 0.20,
    ("Friday", "11:30-1:00 PM"): 0.00,
    
    
    ("Monday", "1:00-2:30 PM"): 0.20,
    ("Tuesday", "1:00-2:30 PM"): 0.20,
    ("Wednesday", "1:00-2:30 PM"): 0.25,
    ("Thursday", "1:00-2:30 PM"): 0.20,
    ("Friday", "1:00-2:30 PM"): 0.20,
    
    
    ("Monday", "2:30-4:00 PM"): 0.20,
    ("Tuesday", "2:30-4:00 PM"): 0.20,
    ("Wednesday", "2:30-4:00 PM"): 0.00,  
    ("Thursday", "2:30-4:00 PM"): 0.20,
    ("Friday", "2:30-4:00 PM"): 0.20,
    
    ("Monday", "4:00-5:30 PM"): 0.20,
    ("Tuesday", "4:00-5:30 PM"): 0.20,
    ("Wednesday", "4:00-5:30 PM"): 0.00,  
    ("Thursday", "4:00-5:30 PM"): 0.20,
    ("Friday", "4:00-5:30 PM"): 0.20,
    
    # Evening slots
    ("Monday", "6:00-7:30 PM"): 0.20,
    ("Tuesday", "6:00-7:30 PM"): 0.20,
    ("Wednesday", "6:00-7:30 PM"): 0.20,
    ("Thursday", "6:00-7:30 PM"): 0.20,
    ("Friday", "6:00-7:30 PM"): 0.20,
    
    ("Monday", "7:30-9:00 PM"): 0.20,
    ("Tuesday", "7:30-9:00 PM"): 0.20,
    ("Wednesday", "7:30-9:00 PM"): 0.20,
    ("Thursday", "7:30-9:00 PM"): 0.20,
    ("Friday", "7:30-9:00 PM"): 0.20
}
variables = {}
model = Model("Scheduling")


# Add slack variables and update the objective to minimize slack
# Add slack variables and update the objective to minimize slack
slack_vars = {}
total_slack = 0  # Initialize total slack



# Define the binary variables
for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    num_sections = int(row['# Sections'])

    if num_sections == 0:
        continue

    # Adjust for "CS435" to have 3 parts instead of 2
    if course == "CS435":
        parts = [1, 2, 3]
    else:
        parts = [1, 2]

    for section_id in range(1, num_sections + 1):
        for part in parts:  # Adjusted parts based on the course
            for day in days:
                for slot in time_slots:
                    var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                    variables[var_name] = model.addVar(vtype=GRB.BINARY, name=var_name)


Aggregated DataFrame:
    Course          Instructor Course_Number              Email  # Sections
0    CS114   Kapleau, Jonathan           114   kapleau@njit.edu           1
1    CS114  Zaidenberg, Ayelet           114      acz6@njit.edu           2
2    CS116             Wu, Jun           116      jw65@njit.edu           2
3    CS241     Ionescu, Adrian           241   ionescu@njit.edu           1
4    CS241       Naik, Kamlesh           241      krn9@njit.edu           1
..     ...                 ...           ...                ...         ...
106  IT340     Senesy, Stanley           340    senesy@njit.edu           2
107  IT342    Giannoglou, Karl           342     kg338@njit.edu           1
108  IT420    Grayson, Tanisha           420  tgrayson@njit.edu           1
109  IT430     Senesy, Stanley           430    senesy@njit.edu           1
110  IT490       Patel, Dipesh           490   pateldi@njit.edu           2

[111 rows x 5 columns]
Set parameter Username
Set parameter Lice

In [5]:
# Calculate total number of section parts (2N)
#print("program is working!")
# Read the "pre-scheduled" sheet


# Group by (Day, Time) to count the number of pre-scheduled courses at each (day, time)
pre_scheduled_counts = df_pre_scheduled.groupby(['Day', 'Time']).size().to_dict()

total_section_parts = 2 * sum(int(row['# Sections']) for _, row in df.iterrows()) + df[df["Course"] == "CS435"]["# Sections"].sum()

print("Total section parts:", total_section_parts)
s = 0

for (day, slot), percentage in slot_percentages.items():
    # Find how many courses are pre-scheduled for this (day, slot)
#    x = pre_scheduled_counts.get((day, slot), 0)  # Get the count or 0 if not pre-scheduled

    slot_vars = [variables[var_name] for var_name in variables if var_name.split('_')[5] == day and var_name.split('_')[6] == slot]

    # Adjust max_section_parts_slot by adding the number of pre-scheduled courses (x)
    max_section_parts_slot = math.ceil((percentage / 6) * total_section_parts) 
    s += max_section_parts_slot

    if percentage > 0:  # Only add slack variables if the percentage is non-zero
        model.addConstr(
            sum(slot_vars) <= max_section_parts_slot,
            name=f"balance_slot_with_slack_{day}_{slot}"
        )
    else:
        model.addConstr(
            sum(slot_vars) <= max_section_parts_slot,
            name=f"balance_slot_no_slack_{day}_{slot}"
        )

print("Sum of the balances is:", s)

# Evening slot constraints
evening_percentage = 0.20  # Evening slots defined as 6:00-7:30 PM and 7:30-9:00 PM

evening_slots = ["6:00-7:30 PM", "7:30-9:00 PM"]
for slot in evening_slots:
    evening_vars = [variables[var_name] for var_name in variables if var_name.split('_')[6] == slot]

    # Adjust for pre-scheduled evening slots
    x_evening = pre_scheduled_counts.get(("Friday", slot), 0)  # Example to check pre-scheduled Friday slots (modify as needed)
    max_section_parts_evening = (evening_percentage / 6) * total_section_parts + x_evening

    model.addConstr(sum(evening_vars) <= max_section_parts_evening, name=f"balance_evening_{slot}")




# Read the "pre-scheduled" sheet
df_pre_scheduled = pd.read_excel(excel_file, sheet_name='pre-scheduled')

print(df_pre_scheduled.columns)


# Enforce pre-scheduled courses

# Read the "pre-scheduled" sheet


# Enforce that no other courses for this instructor can be scheduled at the specified (day, time)
for _, row in df_pre_scheduled.iterrows():
    instructor = row['Instructor']
    day = row['Day']
    times = row['Time']

    # Set all variables for this (instructor, day, time) to 0
    for var_name in variables:
        var_parts = var_name.split('_')
        var_instructor = var_parts[2]
        var_day = var_parts[5]
        var_time = var_parts[6]

        # Check if the variable matches the instructor, day, and time
        if var_instructor == instructor and var_day == day and var_time == times:
            model.addConstr(variables[var_name] == 0, name=f"block_{instructor}_{day}_{times}")



# Set the objective to maximize the number of graduate courses scheduled on consecutive slots



# Add the constraints with numbering and spacing
# Add the constraints with numbering and spacing
constraint_counter = 0

for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    num_sections = int(row['# Sections'])

    if num_sections == 0:
        continue

    # Adjust for "CS435" to have 3 parts instead of 2
    if course == "CS435":
        parts = [1, 2, 3]
    else:
        parts = [1, 2]

    for section_id in range(1, num_sections + 1):
        for part in parts:
            section_vars = []
            for day in days:
                for slot in time_slots:
                    var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                    if var_name in variables:
                        var = variables[var_name]
                        section_vars.append(var)

            if section_vars:
                constraint_name = f"unique_slot_{course}_{instructor}_{section_id}_{part}"
                model.addConstr(sum(section_vars) == 1, name=constraint_name)
                constraint_counter += 1

# Ensure an instructor is not scheduled for more than one section at the same time slot
for instructor in df['Instructor'].unique():
    for day in days:
        for slot in time_slots:
            instructor_vars = []
            for _, row in df[df['Instructor'] == instructor].iterrows():
                course = row['Course']
                num_sections = int(row['# Sections'])
                if course == "CS435":
                    parts = [1, 2, 3]
                else:
                    parts = [1, 2]
                for section_id in range(1, num_sections + 1):
                    for part in parts:
                        var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                        if var_name in variables:
                            var = variables[var_name]
                            instructor_vars.append(var)

            if instructor_vars:
                constraint_name = f"one_section_per_slot_{instructor}_{day}_{slot}"
                model.addConstr(sum(instructor_vars) <= 1, name=constraint_name)


Total section parts: 298
Sum of the balances is: 382
Index(['Course', 'Section ', 'Instructor', 'Capacity', 'Day', 'Time'], dtype='object')


In [6]:
import gurobipy as gp

# Define the specific time slots for the constraint
restricted_time_slots = ["8:30-10:00 AM", "10:00-11:30 AM", "6:00-7:30 PM", "7:30-9:00 PM"]

# Add constraint for each instructor on each day
for instructor in df['Instructor'].unique():
    for day in days:
        # Collect the binary variables corresponding to the restricted time slots
        restricted_vars = []
        for course in df['Course'].unique():
            instructor_courses = df[(df['Instructor'] == instructor) & (df['Course'] == course)]

            for _, course_row in instructor_courses.iterrows():
                num_sections = int(course_row['# Sections'])
                for section_id in range(1, num_sections + 1):
                    parts = [1, 2] if course != "CS435" else [1, 2, 3]
                    for part in parts:
                        for slot in restricted_time_slots:
                            var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                            if var_name in variables:
                                restricted_vars.append(variables[var_name])

        # Add constraint to ensure that at most 3 of the 4 restricted slots can be assigned
        model.addConstr(
            gp.quicksum(restricted_vars) <= 3,
            name=f"restricted_time_slots_{instructor}_{day}"
        )


In [7]:
# Define valid start times for graduate-style patterns
valid_start_times = ["8:30-10:00 AM", "6:00-7:30 PM"]

# Define additional valid start times for Fridays
friday_start_times = ["8:30-10:00 AM", "1:00-2:30 PM", "2:30-4:00 PM", "4:00-5:30 PM", "6:00-7:30 PM", "7:30-9:00 PM"]

for course in df['Course'].unique():
    for instructor in df['Instructor'].unique():
        course_instructor_rows = df[(df['Course'] == course) & (df['Instructor'] == instructor)]
        if course_instructor_rows.empty:
            continue

        for section_id in range(1, course_instructor_rows.iloc[0]['# Sections'] + 1):
            compatible_pairs = []  # Track compatible (day1, slot1) and (day2, slot2) pairs
            grad_var = model.addVar(vtype=GRB.BINARY, name=f"Grad_{course}_{instructor}_{section_id}")
            undergrad_var = model.addVar(vtype=GRB.BINARY, name=f"Undergrad_{course}_{instructor}_{section_id}")

            y_var_dict = {}  # Store Y variables for quick lookup

            for day1 in days:
                for slot1 in time_slots:
                    for day2 in days:
                        for slot2 in time_slots:
                            # Define the binary variable for this (day1, slot1), (day2, slot2) pair
                            y_var_name = f"Y_{course}_{instructor}_{section_id}_{day1}_{slot1}_{day2}_{slot2}"
                            y_var = model.addVar(vtype=GRB.BINARY, name=y_var_name)
                            compatible_pairs.append(y_var)
                            y_var_dict[(day1, slot1, day2, slot2)] = y_var

                            var_part1 = f"X_{course}_{instructor}_{section_id}_1_{day1}_{slot1}"
                            var_part2 = f"X_{course}_{instructor}_{section_id}_2_{day2}_{slot2}"

                            # Graduate pattern (consecutive slots on the same day)
                            if day1 == day2:
                                if day1 == "Friday":
                                    # Friday-specific constraint: Allow 8:30 AM and 1:00 PM or later
                                    if slot1 in friday_start_times and time_slots.index(slot2) == time_slots.index(slot1) + 1:
                                        model.addConstr(y_var <= grad_var, name=f"grad_pair_friday_{course}_{instructor}_{section_id}_{day1}_{slot1}_{slot2}")
                                    else:
                                        # Disable non-consecutive or invalid start times for Fridays
                                        model.addConstr(y_var == 0, name=f"invalid_grad_friday_{course}_{instructor}_{section_id}_{day1}_{slot1}_{slot2}")
                                else:
                                    # Other days: Allow only 8:30 AM or 6:00 PM starts
                                    if slot1 in valid_start_times and time_slots.index(slot2) == time_slots.index(slot1) + 1:
                                        model.addConstr(y_var <= grad_var, name=f"grad_pair_non_friday_{course}_{instructor}_{section_id}_{day1}_{slot1}_{slot2}")
                                    else:
                                        # Disable non-consecutive or invalid start times for other days
                                        model.addConstr(y_var == 0, name=f"invalid_grad_non_friday_{course}_{instructor}_{section_id}_{day1}_{slot1}_{slot2}")

                            # Undergraduate pattern (same slot, different valid day pairs)
                            elif day1 != day2 and slot1 == slot2:
                                if (day1 == "Monday" and day2 in ["Wednesday", "Thursday"]) or \
                                   (day1 == "Tuesday" and day2 in ["Thursday", "Friday"]) or \
                                   (day1 == "Wednesday" and day2 == "Friday"):
                                    model.addConstr(y_var <= undergrad_var, name=f"undergrad_pair_enforce_{course}_{instructor}_{section_id}_{day1}_{slot1}_{day2}_{slot2}")
                                else:
                                    model.addConstr(y_var == 0, name=f"invalid_undergrad_pair_{course}_{instructor}_{section_id}_{day1}_{slot1}_{day2}_{slot2}")
                            else:
                                model.addConstr(y_var == 0, name=f"invalid_pair_{course}_{instructor}_{section_id}_{day1}_{slot1}_{day2}_{slot2}")

            # Ensure Part 1 of courses where the third character is "7" cannot be scheduled from 8:30 to 10:00 AM
            for day1 in days:
                for slot1 in time_slots:
                    if course[2] == "7" and slot1 == "8:30-10:00 AM":
                        var_part1 = f"X_{course}_{instructor}_{section_id}_1_{day1}_{slot1}"
                        if var_part1 in variables:
                            model.addConstr(variables[var_part1] == 0, name=f"no_8_30_to_10_CS7XX_{course}_{instructor}_{section_id}_{day1}_{slot1}")

            # Link Part 1 variables to the sum of corresponding Y variables
            for day1 in days:
                for slot1 in time_slots:
                    var_part1 = f"X_{course}_{instructor}_{section_id}_1_{day1}_{slot1}"
                    if var_part1 in variables:
                        y_vars_for_part1 = [y_var_dict[(day1, slot1, day2, slot2)] for day2 in days for slot2 in time_slots]
                        model.addConstr(sum(y_vars_for_part1) == variables[var_part1], name=f"part1_link_{course}_{instructor}_{section_id}_{day1}_{slot1}")

            # Link Part 2 variables to the sum of corresponding Y variables
            for day2 in days:
                for slot2 in time_slots:
                    var_part2 = f"X_{course}_{instructor}_{section_id}_2_{day2}_{slot2}"
                    if var_part2 in variables:
                        y_vars_for_part2 = [y_var_dict[(day1, slot1, day2, slot2)] for day1 in days for slot1 in time_slots]
                        model.addConstr(sum(y_vars_for_part2) == variables[var_part2], name=f"part2_link_{course}_{instructor}_{section_id}_{day2}_{slot2}")

            # Ensure exactly one (day1, slot1), (day2, slot2) pair is selected
            model.addConstr(sum(compatible_pairs) == 1, name=f"select_one_pair_{course}_{instructor}_{section_id}")

            # Ensure that only one pattern (graduate or undergraduate) is chosen
            model.addConstr(grad_var + undergrad_var == 1, name=f"select_one_pattern_{course}_{instructor}_{section_id}")




In [8]:
restricted_day = "Monday"
restricted_time_slot = "4:00-5:30 PM"

# Add the constraint: Only courses with course number > 199 and "Sum of C" < 35 can be scheduled on Monday from 4:00 to 5:30
for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    course_number = int(row['Course_Number'])
    for sc in range(1,row['# Sections']+1):
        capacity =  section_capacity_map.get((course, instructor, sc))

    # Check if the course meets the condition for being scheduled in this restricted slot
        if course_number > 199 and capacity < 35:
        # Loop over sections and parts to ensure the variables for this course are allowed to be scheduled
            for section_id in range(1, int(row['# Sections']) + 1):
                if course == "CS435":
                    parts = [1, 2, 3]
                else:
                    parts = [1, 2]
                for part in parts:
                    var_name = f"X_{course}_{instructor}_{section_id}_{part}_{restricted_day}_{restricted_time_slot}"
                    if var_name in variables:
                    # No constraint needed, the course meets the condition
                        continue
        else:
        # If the course does not meet the conditions, add a constraint to prevent it from being scheduled in the restricted slot
            for section_id in range(1, int(row['# Sections']) + 1):
                if course == "CS435":
                    parts = [1, 2, 3]
                else:
                    parts = [1, 2]
                for part in parts:
                    var_name = f"X_{course}_{instructor}_{section_id}_{part}_{restricted_day}_{restricted_time_slot}"
                    if var_name in variables:
                        model.addConstr(variables[var_name] == 0, name=f"restricted_slot_{course}_{instructor}_{section_id}_{part}_{restricted_day}_{restricted_time_slot}")

# Set slack variables to zero manually
#for slack_var_name, slack_var in slack_vars.items():
#    model.addConstr(slack_var == 0, name=f"set_{slack_var_name}_to_zero")
# Define the course blocks


# Define course blocks and special blocks with <= 2 constraints
course_blocks = [
    ['CS114', 'IS210', 'CS450', 'CS337'],
    ['CS241', 'CS280', 'IS350'],
    ['CS288', 'CS332', 'CS301', 'CS356'],  # Special block (<= 2)
    ['CS341', 'CS350', 'CS351', 'CS331', 'CS375'],  # Special block (<= 2)
    ['CS435', 'CS490', 'CS485', 'CS370', 'CS375'],
    ['CS485', 'CS491', 'CS450', 'CS482'],
    ['CS610', 'CS630', 'CS631', 'CS656', 'DS675', 'CS675', 'CS670'],  # Block-grad-core
    ['DS677', 'DS669', 'DS650', 'CS670', 'CS610', 'CS665', 'CS667', 'CS732', 'DS680'],  # Block-grad-DS+Alg
    ['CS608', 'CS645', 'CS646', 'CS647', 'CS648', 'CS678', 'CS696'],  # Block-grad-cyber
    ['IS455', 'IS645'],
    ['IT220', 'IT230', 'IT240', 'IT302'],
    ['IT256', 'IT266', 'IT286', 'IT360', 'IT380', 'IT383', 'IT386'],
    ['IT120', 'IT240']
]

# List of special blocks with <= 2 constraints
special_blocks = [
    ['CS288', 'CS332', 'CS301', 'CS356'],
    ['CS341', 'CS350', 'CS351', 'CS331', 'CS375']
]

# Add constraint to ensure no different parts of courses in the same block are scheduled in the same day and time slot
for block in course_blocks:
    for course1 in block:
        for course2 in block:
            if course1 != course2:
                for instructor1 in df[df['Course'] == course1]['Instructor'].unique():
                    for instructor2 in df[df['Course'] == course2]['Instructor'].unique():
                        for day in days:
                            for slot in time_slots:
                                var_course1_part1 = f"X_{course1}_{instructor1}_1_{day}_{slot}"
                                var_course1_part2 = f"X_{course1}_{instructor1}_2_{day}_{slot}"
                                var_course2_part1 = f"X_{course2}_{instructor2}_1_{day}_{slot}"
                                var_course2_part2 = f"X_{course2}_{instructor2}_2_{day}_{slot}"

                                # Check if the current block is a special block (<= 2 constraints)
                                max_constraint = 2 if block in special_blocks else 1

                                # Add constraints for part 1
                                if var_course1_part1 in variables and var_course2_part1 in variables:
                                    model.addConstr(
                                        variables[var_course1_part1] + variables[var_course2_part1] <= max_constraint,
                                        name=f"no_same_day_slot_{course1}_{instructor1}_{course2}_{instructor2}_{day}_{slot}_part1"
                                    )
                                # Add constraints for part 2
                                if var_course1_part2 in variables and var_course2_part2 in variables:
                                    model.addConstr(
                                        variables[var_course1_part2] + variables[var_course2_part2] <= max_constraint,
                                        name=f"no_same_day_slot_{course1}_{instructor1}_{course2}_{instructor2}_{day}_{slot}_part2"
                                    )

                                    

In [9]:
part1_slot = "6:00-7:30 PM"
part2_slot = "7:30-9:00 PM"
    
    # Iterate over the DataFrame and add constraints for each section
for _, row in df.iterrows():
    course = row['Course']
    instructor = row['Instructor']
    num_sections = int(row['# Sections'])
    
    if num_sections == 0:
        continue

    for section_id in range(1, num_sections + 1):
        for day in days:
            # Get the variable names for part 1 and part 2 for the relevant time slots
            part1_var_name = f"X_{course}_{instructor}_{section_id}_1_{day}_{part1_slot}"
            part2_var_name = f"X_{course}_{instructor}_{section_id}_2_{day}_{part2_slot}"
            
            # Ensure that the assignment of part 1 at 6:00-7:30 PM equals the assignment of part 2 at 7:30-9:00 PM on the same day
            model.addConstr(variables[part1_var_name] == variables[part2_var_name],
                            name=f"timing_constraint_{course}_{instructor}_{section_id}_{day}")


In [10]:
for instructor in df['Instructor'].unique() :

    # Iterate over all days
    for day in days:
        
        # Iterate over all possible starting time slots (the first of three consecutive slots)
        for i in range(len(time_slots) - 2):
            slot1 = time_slots[i]
            slot2 = time_slots[i + 1]
            slot3 = time_slots[i + 2]
            
            # Initialize a variable to sum the decision variables for the three consecutive slots
            
            
            # Filter the DataFrame to get only the courses and sections taught by the current instructor
            instructor_df = df[df['Instructor'] == instructor]

            # Loop over the first course taught by the instructor
            for idx1, row1 in instructor_df.iterrows():
                course1 = row1['Course']
                num_sections1 = int(row1['# Sections'])
                
                if num_sections1 == 0:
                    continue

                # Loop over the first section and part for the first slot
                for section_id1 in range(1, num_sections1 + 1):
                    if course1 == "CS435":
                        parts = [1, 2, 3]
                    else:
                        parts = [1, 2]
                    for part1 in parts:
                        # Build the variable name for the first slot
                        var_name1 = f"X_{course1}_{instructor}_{section_id1}_{part1}_{day}_{slot1}"

                        # Loop over the second course taught by the instructor
                        for idx2, row2 in instructor_df.iterrows():
                            course2 = row2['Course']
                            num_sections2 = int(row2['# Sections'])
                            
                            if num_sections2 == 0:
                                continue

                            # Loop over the second section and part for the second slot
                            for section_id2 in range(1, num_sections2 + 1):
                                if course1 == "CS435":
                                    parts = [1, 2, 3]
                                else:
                                    parts = [1, 2]
                                for part2 in parts:
                                    # Build the variable name for the second slot
                                    var_name2 = f"X_{course2}_{instructor}_{section_id2}_{part2}_{day}_{slot2}"

                                    # Check if var_name1 and var_name2 are different
                                    if (course1 != course2 or section_id1 != section_id2 or part1 != part2):
                                        
                                        # Loop over the third course taught by the instructor
                                        for idx3, row3 in instructor_df.iterrows():
                                            course3 = row3['Course']
                                            num_sections3 = int(row3['# Sections'])
                                            
                                            if num_sections3 == 0:
                                                continue

                                            # Loop over the third section and part for the third slot
                                            for section_id3 in range(1, num_sections3 + 1):
                                                if course1 == "CS435":
                                                    parts = [1, 2, 3]
                                                else:
                                                    parts = [1, 2]
                                                for part3 in parts:
                                                    # Build the variable name for the third slot
                                                    var_name3 = f"X_{course3}_{instructor}_{section_id3}_{part3}_{day}_{slot3}"

                                                    # Check if var_name3 is different from both var_name1 and var_name2
                                                    if (course1 != course3 or section_id1 != section_id3 or part1 != part3) and \
                                                       (course2 != course3 or section_id2 != section_id3 or part2 != part3):
                                                        consecutive_sum=0

                                                        # Add the variables to the sum if they exist in the dictionary
                                                        if var_name1 in variables:
                                                            consecutive_sum += variables[var_name1]
                                                        if var_name2 in variables:
                                                            consecutive_sum += variables[var_name2]
                                                        if var_name3 in variables:
                                                            consecutive_sum += variables[var_name3]

                                                        # Add the constraint that the sum of these variables must be <= 2
                                                        model.addConstr(consecutive_sum <= 2, 
                                                        name=f"consecutive_slots_constraint_{instructor}_{day}_{slot1}_{slot2}_{slot3}")

 

In [11]:
total_points = 0  # Initialize the total points for the model


time_slot_mapping = {
    'M': 'Monday', 
    'T': 'Tuesday', 
    'W': 'Wednesday', 
    'R': 'Thursday', 
    'F': 'Friday',
    'S': 'Saturday'  # Define 'S' for completeness, but we'll ignore it
}

# Time slot indexes
time_slot_index = {
    '1': "8:30-10:00 AM",
    '2': "10:00-11:30 AM",
    '3': "11:30-1:00 PM",
    '4': "1:00-2:30 PM",
    '5': "2:30-4:00 PM",
    '6': "4:00-5:30 PM",
    '7': "6:00-7:30 PM",
    '8': "7:30-9:00 PM"
}

# Sample data format:
# instructor_constraints = {
#     "ahh2@njit.edu": ["M2", "R2", "M4", "T4", "M5", "T5", "T7", "T8"],
#     "alexg@njit.edu": ["M5", "R3"]
# }

# Add the constraints for instructors who have "Health" or "Religion" type
df_constraints = pd.read_excel(excel_file, sheet_name='Constraints & Preferences')
for _, row in df_constraints.iterrows():
    instructor_info = row['Instructor UCID: Type']
    slots = row['Slots']
    if isinstance(instructor_info, float):
        break

    # Parse the instructor UCID and type
    email, constraint_type = instructor_info.split(": ")

    # We only care about the instructors with "Health" or "Religion" type
    if constraint_type.strip() in ["Health", "Religion"]:
        # Parse the blocked time slots for this instructor
        blocked_slots = slots.split("|")[1:-1]  # Remove empty elements from split

        for slot_code in blocked_slots:
            # Extract the day and time slot
            day_abbrev = slot_code[0]  # M, T, W, R, F
            time_slot_num = slot_code[1]  # 1-8

            day_full = time_slot_mapping[day_abbrev]
            time_slot_full = time_slot_index[time_slot_num]

            # Find the instructor name from the df
            instructor_row = df[df['Email'] == email]

            if not instructor_row.empty:
                instructor_name = instructor_row['Instructor'].iloc[0]  # Get the name as a scalar

                # Add a soft constraint to block this time slot for all parts of the instructor's courses
                for course in df[df['Instructor'] == instructor_name]['Course']:
                    filtered_df = df[df['Course'] == course]

                    if not filtered_df.empty:  # Only proceed if the filtered DataFrame is not empty
                        num_sections = int(filtered_df['# Sections'].iloc[0])

                        for section_id in range(1, num_sections + 1):
                            parts = [1, 2] if course != "CS435" else [1, 2, 3]  # CS435 has 3 parts

                            for part in parts:
                                var_name = f"X_{course}_{instructor_name}_{section_id}_{part}_{day_full}_{time_slot_full}"

                                # Ensure the variable exists in the model
                                if var_name in variables:
                                    slack_var_name = f"Slack_{course}_{instructor_name}_{section_id}_{part}_{day_full}_{time_slot_full}"
                                    slack_var = model.addVar(vtype=GRB.BINARY, name=slack_var_name)

                                    # Add the soft constraint (allow slack)
                                    model.addConstr(
                                        variables[var_name] <= slack_var,
                                        name=f"health_religion_constraint_{instructor_name}_{day_full}_{time_slot_full}"
                                    )

                                    # Subtract 2048 points if the constraint is violated
                                    total_points -= 2048 * slack_var

In [12]:
points_per_slot = {
    "Pref-1": 3,
    "Pref-2": 2,
    "Pref-3": 1
}


instructor_penalty_tracker = {}

instructor_soft_violated=[]

for idx, row in df_constraints.iterrows():
    instructor_info = row['Instructor UCID: Type']
    slots = row['Slots']
    if(isinstance(instructor_info,float)):
        break
    # Parse the instructor UCID and constraint type
    email, constraint_type = instructor_info.split(": ")
    
    # Parse the blocked time slots for this instructor
    blocked_slots = slots.split("|")[1:-1]  # Remove empty elements from split

    # Handle different types of constraints
    if constraint_type.strip() == "Pref-1":
        points = 8
    elif constraint_type.strip() == "Pref-2":
        points = 4
    elif constraint_type.strip() == "Pref-3":
        points = 2
    elif constraint_type.strip() in ["Health", "Religion"]:
        continue  # No points assigned for Health/Religion as these are hard constraints
    elif constraint_type.strip() == "Childcare":
        points=-1024
    else:
        points = -8  # Default negative points for all other types
    for slot_code in blocked_slots:
        try:
            # Check if the length of slot_code is at least 2
            if len(slot_code) < 2:
                print(f"Error in row {idx}, email: {email}, constraint: {constraint_type}, slot code: '{slot_code}' (invalid length)")
                continue  # Skip this slot if it's too short

            # Extract the day and time slot
            day_abbrev = slot_code[0]  # M, T, W, R, F
            time_slot_num = slot_code[1]  # 1-8

            # Extract the full day and time slot
            day_full = time_slot_mapping[day_abbrev]
            time_slot_full = time_slot_index[time_slot_num]

            # Find the instructor's name using the email, skip if not found
            instructor_row = df[df['Email'] == email]
            if instructor_row.empty:
                #print(f"Warning: Instructor with email {email} not found in 'df'. Skipping.")
                continue

            instructor_name = instructor_row['Instructor'].iloc[0]


            # Add or subtract points based on the constraint type for all sections and parts of the instructor's courses
            for course in df[df['Instructor'] == instructor_name]['Course']:

                filtered_df = df[df['Course'] == course]
                if not filtered_df.empty:  # Only proceed if the filtered DataFrame is not empty
                    num_sections = int(filtered_df[(filtered_df['Instructor'] == instructor_name) & (filtered_df['Course'] == course)]['# Sections'].iloc[0])


                    for section_id in range(1, num_sections + 1):
  #                      if instructor_name in instructors_applied:
  #                          break
                        if course == "CS435":
                            parts = [1, 2, 3]
                        else:
                            parts = [1, 2]
                        for part in parts:  # 2 parts per section
                            var_name = f"X_{course}_{instructor_name}_{section_id}_{part}_{day_full}_{time_slot_full}"
                            
                            # Ensure the variable exists in the model
                            if var_name in variables:
                                total_points += points * variables[var_name] 
                                if(points<-1):
                                    instructor_soft_violated.append(var_name)

                                # Log the points added or subtracted
                                action = "Adding" if points > 0 else "Subtracting"
                              #  print(f"{action} {abs(points)} points for scheduling {instructor_name} (email: {email}) "
                               #       f"on {day_full}, time slot {time_slot_full} for course {course}, section {section_id}, "
                                #      f"part {part}, due to constraint type: {constraint_type}")
        except IndexError:
            # Print the row index and the problematic slot cprint(f"Error in row {idx}, email: {email}, constraint: {constraint_type}, slot code: '{slot_code}'")
            continue  # Skip this slot and move to the next one

# Set the objective to maximize the sum of points

#model.addConstr(total_slack == 0, name=f"balance constraints are hard.")

In [13]:
# from gurobipy import Model, GRB

# def add_slack_variables_and_constraints(df, model, variables):
#     """
#     This function defines slack variables u_(instructor, day) and adds constraints to ensure
#     that u_(instructor, day) is 1 if the instructor is assigned to work on that day.
    
#     Parameters:
#         df (pd.DataFrame): The DataFrame containing course, instructor, and section data.
#         model (gurobipy.Model): The Gurobi model.
#         variables (dict): Dictionary of Gurobi variables representing the assignment of
#                           instructors to sections, days, and time slots.
                          
#     Returns:
#         dict: A dictionary of slack variables u_(instructor, day).
#     """
#     slack_vars = {}

#     # Loop through the DataFrame to create slack variables for each (instructor, day)
#     for _, row in df.iterrows():
#         course = row['Course']
#         instructor = row['Instructor']
#         num_sections = int(row['# Sections'])
#         if num_sections == 0:
#             continue

#         for day in days:
#             # Define a slack variable u_(instructor, day) for each instructor and day
#             slack_var_name = f"u_{instructor}_{day}"
#             if slack_var_name not in slack_vars:
#                 slack_vars[slack_var_name] = model.addVar(vtype=GRB.BINARY, name=slack_var_name)

#             # Add constraints to ensure that u_(instructor, day) is >= any assigned variables for that day
#             for section_id in range(1, num_sections + 1):
#                 if course == "CS435":
#                     parts = [1, 2, 3]
#                 else:
#                     parts = [1, 2]
#                 for part in parts:  # Two parts per section
#                     for slot in time_slots:
#                         var_name = f"X_{row['Course']}_{instructor}_{section_id}_{part}_{day}_{slot}"
#                         if var_name in variables:
#                         # Ensure u_(instructor, day) is 1 if any X_(course, instructor, section, part, day, slot) is 1
#                             model.addConstr(variables[var_name] <= slack_vars[slack_var_name], 
#                                         name=f"slack_constraint_{instructor}_{day}_{section_id}_{part}_{slot}")

#     return slack_vars
# def compute_sum_of_slack_variables(slack_vars, model):
#     """
#     This function computes the sum of all slack variables u_(instructor, day).
    
#     Parameters:
#         slack_vars (dict): Dictionary of slack variables u_(instructor, day).
#         model (gurobipy.Model): The Gurobi model.
    
#     Returns:
#         float: The sum of all slack variables.
#     """
#     # Compute the total sum of all slack variables (sum of u_(instructor, day))
#     total_slack_sum = model.addVar(vtype=GRB.CONTINUOUS, name="total_slack_sum")
#     model.addConstr(total_slack_sum == sum(slack_vars.values()), name="sum_slack_constraint")
    
#     return total_slack_sum
# slack_vars = add_slack_variables_and_constraints(df, model, variables)

In [14]:
# Read the General Preferences sheet
from gurobipy import LinExpr

# Initialize a penalty expression for violated preferences
format_penalty_sum = LinExpr()

# Iterate over instructor preferences and add penalties for violations
penalty_value = -8  # Penalty for violating the format preference

# Read the Excel file
general_preferences_df = pd.read_excel(excel_file, sheet_name="General Preferences")

# Rename the relevant columns for ease of use
general_preferences_df = general_preferences_df.rename(columns={general_preferences_df.columns[1]: 'Email', general_preferences_df.columns[2]: 'Preference'})


for _, row in general_preferences_df.iterrows():
    email = row['Email']
    preference = row['Preference']

    # Find the instructor's name from the main DataFrame using the email
    instructor_row = df[df['Email'] == email]
    if instructor_row.empty:
        continue

    instructor_name = instructor_row['Instructor'].iloc[0]

    # Filter DataFrame for courses assigned to the instructor
    instructor_courses = df[df['Instructor'] == instructor_name]

    for _, course_row in instructor_courses.iterrows():
        course = course_row['Course']
        num_sections = int(course_row['# Sections'])

        for section_id in range(1, num_sections + 1):
            parts = [1, 2] if course != "CS435" else [1, 2, 3]

            # Iterate over pairs of time slots
            for day in days:
                for i in range(len(time_slots) - 1):  # Ensure next slot exists
                    slot1 = time_slots[i]
                    slot2 = time_slots[i + 1]

                    part1_var_name = f"X_{course}_{instructor_name}_{section_id}_1_{day}_{slot1}"
                    part2_var_name = f"X_{course}_{instructor_name}_{section_id}_2_{day}_{slot2}"

                    if part1_var_name in variables and part2_var_name in variables:
                        part1_var = variables[part1_var_name]
                        part2_var = variables[part2_var_name]

                        # Add penalty variables for violations
                        penalty_var = model.addVar(vtype=GRB.BINARY, name=f"Penalty_{instructor_name}_{day}_{slot1}_{slot2}")

                        if preference == "3-hour format":
                            # Add penalty for non-consecutive parts (violation of 3-hour format preference)
                            model.addConstr(part1_var - part2_var <= penalty_var, name=f"violation_consecutive_{course}_{instructor_name}_{section_id}_{day}_{slot1}_{slot2}")
                            model.addConstr(part2_var - part1_var <= penalty_var, name=f"violation_consecutive_{course}_{instructor_name}_{section_id}_{day}_{slot1}_{slot2}")

                        elif preference == "1.5+1.5 hour format":
                            # Add penalty for assigning consecutive parts (violation of non-consecutive preference)
                            model.addConstr(part1_var + part2_var - penalty_var <= 1, name=f"violation_non_consecutive_{course}_{instructor_name}_{section_id}_{day}_{slot1}_{slot2}")

                        # Add the penalty to the sum
                        format_penalty_sum += penalty_value * penalty_var


In [15]:
general_preferences_df = pd.read_excel(excel_file, sheet_name="General Preferences")
general_preferences_df = general_preferences_df.rename(columns={general_preferences_df.columns[1]: 'Email', general_preferences_df.columns[3]: 'Day Preference'})

# Create a dictionary mapping emails to their day preferences
day_preference_dict = general_preferences_df.set_index('Email')['Day Preference'].to_dict()


z_vars = {}
for instructor in df['Instructor'].unique():
    for day in days:
        # Binary variable indicating whether the instructor teaches on that day
        z_var = model.addVar(vtype=GRB.BINARY, name=f"Z_{instructor}_{day}")
        z_vars[(instructor, day)] = z_var

        # Relevant X variables for this instructor and day
        relevant_x_vars = []
        for course in df['Course'].unique():
            course_instructor_rows = df[(df['Course'] == course) & (df['Instructor'] == instructor)]

            if course_instructor_rows.empty:
                continue

            num_sections = course_instructor_rows['# Sections'].iloc[0]
            for section_id in range(1, num_sections + 1):
                for part in [1, 2]:
                    for slot in time_slots:
                        x_var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                        if x_var_name in variables:
                            x_var = variables[x_var_name]
                            relevant_x_vars.append(x_var)
                            # Ensure that if X variable is 1, z_var must be 1
                            model.addConstr(
                                x_var <= z_var,
                                name=f"x_var_link_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                            )

        # If no relevant X variables are found, ensure that z_var is set to 0
        if not relevant_x_vars:
            model.addConstr(z_var == 0, name=f"no_classes_{instructor}_{day}")

# Calculate penalty sum S
day_penalty_sum = LinExpr()

# Iterate over instructors and days to add penalties based on their preferences
for instructor in df['Instructor'].unique():
    # Get the email of the instructor
    email = df[df['Instructor'] == instructor]['Email'].iloc[0]

    # Check if the instructor prefers condensed days
    prefers_condensed_days = day_preference_dict.get(email, "No") == "I prefer to condense my sections into fewer days"

    # Set the penalty value
    penalty_value = -8 if prefers_condensed_days else -3

    for day in days:
        # Binary variable indicating whether the instructor teaches on that day
        z_var = z_vars[(instructor, day)]  # This should now always exist

        # Add penalty to the penalty sum
        day_penalty_sum += penalty_value * z_var




In [16]:
from gurobipy import LinExpr

preferences_df = pd.read_excel(excel_file, sheet_name="General Preferences")

# Create a dictionary mapping instructor emails to their preference for consecutive slots
email_column_name = preferences_df.columns[1]  # Get the name of the second column (emails)
consecutive_preference_column_name = preferences_df.columns[5]  # Assuming the 6th column is for consecutive slots

# Create a dictionary mapping emails to consecutive slot preference
consecutive_preference = preferences_df.set_index(email_column_name)[consecutive_preference_column_name].to_dict()
print(consecutive_preference)

# Initialize a penalty expression to sum penalties
consecutive_penalty_sum = LinExpr()

# Iterate over instructor emails and add penalties for consecutive slots if they dislike them
penalty_value = -2048  # Penalty for consecutive slots

for email, prefers_consecutive in consecutive_preference.items():
    if prefers_consecutive == "No":  # Assuming "No" indicates they dislike consecutive slots
        # Get the instructor's name using the email from the DataFrame
        instructor_row = df[df['Email'] == email]
        if instructor_row.empty:
            continue
        instructor_name = instructor_row['Instructor'].iloc[0]  # Extract the name of the instructor

        for day in days:
            for slot_idx in range(len(time_slots) - 1):
                slot1 = time_slots[slot_idx]
                slot2 = time_slots[slot_idx + 1]
                
                for course in df['Course'].unique():
                    relevant_rows = df[(df['Instructor'] == instructor_name) & (df['Course'] == course)]
               #     print(email, prefers_consecutive)
                    if relevant_rows.empty:
                        continue
                #    print(instructor_name)
                    num_sections = int(relevant_rows['# Sections'].iloc[0])
                    for section_id in range(1, num_sections + 1):
                        # Define X variables for consecutive slots
                        x_var1_name = f"X_{course}_{instructor_name}_{section_id}_1_{day}_{slot1}"
                        x_var2_name = f"X_{course}_{instructor_name}_{section_id}_2_{day}_{slot2}"
                        
                        if x_var1_name in variables and x_var2_name in variables:
                            x_var1 = variables[x_var1_name]
                            x_var2 = variables[x_var2_name]

                            # Add penalty variable
                            penalty_var = model.addVar(vtype=GRB.BINARY, name=f"Penalty_{instructor_name}_{day}_{slot1}_{slot2}")
                            model.addConstr(x_var1 + x_var2 - 2 * penalty_var <= 1, name=f"consecutive_penalty_{instructor_name}_{day}_{slot1}_{slot2}")
                            
                            # Add penalty to the total penalty sum
                            consecutive_penalty_sum += penalty_value * penalty_var
print("done")



{'kapleau@njit.edu': "I don't have a preference", 'rt494@njit.edu': 'Yes', 'ejt25@njit.edu': 'Yes', 'tweiss@njit.edu': 'Yes', 'ko89@njit.edu': "I don't have a preference", 'kp759@njit.edu': "I don't have a preference", 'js9@njit.edu': 'Yes', 'wohn@njit.edu': "I don't have a preference", 'egan@njit.edu': 'Yes', 'mjk76@njit.edu': "I don't have a preference", 'marvin@njit.edu': "I don't have a preference", 'sz457@njit.edu': "I don't have a preference", 'jingli@njit.edu': 'Yes', 'cliu@njit.edu': nan, 'kehoed@njit.edu': 'Yes', 'yl935@njit.edu': 'Yes', 'daher@njit.edu': 'Yes', 'ss797@njit.edu': 'Yes', 'azi3@njit.edu': 'Yes', 'jw65@njit.edu': 'Yes', 'jq55@njit.edu': 'Yes', 'nn43@njit.edu': 'Yes', 'meh43@njit.edu': "I don't have a preference", 'dli@njit.edu': 'No', 'skumar@njit.edu': 'Yes', 'monogiou@njit.edu': 'No', 'itani@njit.edu': 'Yes', 'mx6@njit.edu': "I don't have a preference", 'usman@njit.edu': 'Yes', 'chasewu@njit.edu': 'Yes', 'geller@njit.edu': "I don't have a preference", 'amr239@n

In [17]:
# total_slack_sum = compute_sum_of_slack_variables(slack_vars, model)

# Iterate over all unique instructors



model.setObjective(total_points+consecutive_penalty_sum+format_penalty_sum+day_penalty_sum, GRB.MAXIMIZE)

                       
# Update the model
model.update()

print("scheduling model time= ",time.time()-start_time," seconds\n")
start_optimize=time.time()
#model.setParam("TimeLimit", 60)  # Time limit in seconds (2 minutes)

# Solve the model
model.optimize()

print("Values of z_vars (instructor teaching days):")
for (instructor, day), z_var in z_vars.items():
    print(f"Instructor: {instructor}, Day: {day}, Z_var: {z_var.X:.0f}")
print("optimization time= ",time.time()-start_optimize," seconds\n")

print("\nViolated Slack Variables (costing -2048 points):\n")
violated_slacks = []
for v in model.getVars():
        if "Slack_" in v.varName and v.x > 0.5:  # Check for slack variables with value 1 (indicating a violation)
            print(f"{v.varName}: Value {v.x}")
            violated_slacks.append(v.varName)

if not violated_slacks:
        print("No constraints related to health/religion were violated.")
else:
        print(f"\nTotal number of violated slack variables: {len(violated_slacks)}")


scheduling model time=  77.78138327598572  seconds

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 441491 rows, 252759 columns and 1563022 nonzeros
Model fingerprint: 0x8e137fe3
Variable types: 0 continuous, 252759 integer (252759 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 387348 rows and 240780 columns
Presolve time: 0.42s
Presolved: 54143 rows, 11979 columns, 171210 nonzeros
Variable types: 0 continuous, 11979 integer (11979 binary)
Performing another presolve...
Presolve removed 38307 rows and 3419 columns
Presolve time: 0.39s

Root relaxation: objective -5.235926e+03, 9373 iterations, 1.13 seconds (1.09 work units)

    Nodes    |    Current Node    |    

In [18]:
from datetime import datetime

# Get current date and time
now = datetime.now()

# Format the date and time as a string
date_time_str = now.strftime("%Y-%m-%d %H:%M:%S")


def calculate_scheduled_percentages(df, variables, model):
    """
    This function calculates the percentage of scheduled classes on each day and time slot after optimization.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing course, instructor, and section data.
        variables (dict): Dictionary of Gurobi binary variables representing the assignment of
                          instructors to sections, days, and time slots.
        model (gurobipy.Model): The Gurobi model containing the optimized variables.
    
    Returns:
        pd.DataFrame: A DataFrame containing the percentage of scheduled classes for each day and time slot.
    """
    # Create a dictionary to count the number of scheduled classes for each (day, time slot)
    schedule_counts = { (day, slot): 0 for day in days for slot in time_slots }
    
    total_classes = 0  # Track total number of classes

    # Iterate over the DataFrame to check which variables are active (scheduled)
    for _, row in df.iterrows():
        course = row['Course']
        instructor = row['Instructor']
        num_sections = int(row['# Sections'])
        
        if num_sections == 0:
            continue

        for section_id in range(1, num_sections + 1):
            if course == "CS435":
                parts = [1, 2, 3]
            else:
                parts = [1, 2]
            for part in parts:  # Two parts per section
                for day in days:
                    for slot in time_slots:
                        var_name = f"X_{course}_{instructor}_{section_id}_{part}_{day}_{slot}"
                        
                        # If the variable is scheduled (value is 1)
                        if variables[var_name].X > 0.5:
                            # Increment the count for that (day, slot)
                            schedule_counts[(day, slot)] += 1
                            total_classes += 1

    # Create a DataFrame to store the percentages for each (day, slot)
    percentages = []
    for day, slot in schedule_counts:
        count = schedule_counts[(day, slot)]
        percentage = (count / total_classes) * 100 if total_classes > 0 else 0
        percentages.append({'Day': day, 'Time Slot': slot, 'Percentage': percentage})

    # Convert the list of percentages to a DataFrame
    percentage_df = pd.DataFrame(percentages)
    
    return percentage_df


if model.Status == GRB.INFEASIBLE:
    # If the model is infeasible, compute the IIS and write the ILP model to a text file
    print("The model is infeasible; computing IIS")
    model.computeIIS()

    with open("model_with_constraints"+date_time_str+".txt", "w") as f:
        f.write("Infeasible Model with Numbered Constraints\n\n")
        for i, constr in enumerate(model.getConstrs(), 1):
            if constr.IISConstr:
                f.write(f"Constraint {i}: {constr.ConstrName}\n")
                f.write(f"{model.getRow(constr)} = {constr.RHS}\n\n")

    print("IIS written to model_with_constraints"+date_time_str+".txt")

else:
    for v in model.getVars():
        if(v.x>0.5):
            print(f"{v.VarName}, {v.X}\n")  # Write variable name and its optimized value
    # If the model is feasible, extract and write the schedule to a text file
    schedule = []
    slack_values = []
    
    # Dictionary to track section numbers for each course
    course_section_tracker = {}

    for var in model.getVars():
        if var.varName.startswith('Slack_'):
            # Store slack variables and their values
            slack_values.append((var.varName, var.x))
        elif var.x > 0.5:  # If the variable is selected in the optimal solution
            name = var.varName
            try:
                _, course, instructor, section_id, part, day, slot = name.split('_')
                # Find the corresponding email for the instructor in the DataFrame
                email = df.loc[df['Instructor'] == instructor, 'Email'].values[0]  # Assuming 'Email' column contains the emails
                schedule.append((course, instructor, email, section_id, part, day, slot))
            except ValueError:
                # Handle any variables that may not follow the expected naming pattern
                pass
    
    schedule.sort()

    # Write the lexicographically sorted schedule and slack values to a file
    with open("final_schedule_sorted_by_course "+date_time_str+" .txt", "w") as f:
        f.write("Course Schedule (Lexicographically Sorted):\n\n")
        for entry in schedule:
            course, instructor, email, section_id, part, day, slot = entry
            course = course.strip()
            instructor = instructor.strip()

            # Track the section number for each course
            if course not in course_section_tracker:
                course_section_tracker[course] = 1  # Initialize section number for this course
            
            # Assign the next available section number for this course
            assigned_section_number = course_section_tracker[course]
  # Increment section number for next time
            
            # Retrieve the capacity from the map
            capacity = section_capacity_map.get((course, instructor, int(section_id)))

            # Write the information to the file
            f.write(f"Course: {course}, Instructor: {instructor}, Email: {email}, Section: {assigned_section_number}, Part: {part}, Day: {day}, Slot: {slot}, Capacity: {capacity}\n")
            if((part=="2" and course!="CS435") or (part=="3" and course=="CS435")):
                course_section_tracker[course] += 1
                f.write(f"\n")

    print("Final schedule with slack values written to final_schedule_sorted_by_course.txt")

    # Sort the schedule based on the instructor's name
    schedule_sorted_by_instructor = sorted(schedule, key=lambda x: x[1])  # Sort by the second element (instructor)

    # Write the instructor-sorted schedule and slack values to a separate file, grouped by instructor
    with open("final_schedule_sorted_by_instructor "+date_time_str+".txt", "w") as f:
        f.write("Course Schedule (Sorted by Instructor):\n\n")
        course_section_tracker = {}
        current_instructor = None
        for entry in schedule_sorted_by_instructor:
            course, instructor, email, section_id, part, day, slot = entry
          #  print(type(part))
            # Track the section number for each course (again for this output)
            if course not in course_section_tracker:
                course_section_tracker[course] = 1  # Initialize section number for this course
                
            assigned_section_number = course_section_tracker[course]
            # Increment section number for next time

            # If we encounter a new instructor, print their email and name first
            if instructor != current_instructor:
                if current_instructor is not None:
                    f.write("\n")  # Separate different instructors' sections
                
                f.write(f"Instructor: {instructor}, Email: {email}\n")
                current_instructor = instructor

            capacity = section_capacity_map.get((course, instructor, int(section_id)))

            # Write the course details for the current instructor
            f.write(f"\tCourse: {course}, Section: {assigned_section_number}, Part: {part}, Day: {day}, Slot: {slot}, Capacity: {capacity}\n")
            if((part=="2" and course!="CS435") or (part=="3" and course=="CS435")):
                course_section_tracker[course] += 1

    print("Final schedule sorted by instructor written to final_schedule_sorted_by_instructor.txt")
    percentage_df = calculate_scheduled_percentages(df, variables, model)

# Print the resulting DataFrame with percentages
    print(percentage_df)

# Optionally, write the percentages to a CSV file
    percentage_df.to_csv("scheduled_percentages "+date_time_str+".csv", index=False)
   

# Step 1: Track the days each instructor is scheduled
    instructor_days = defaultdict(set)  # Using a set to ensure uniqueness of days
    
    for entry in schedule:
        course, instructor, email, section_id, part, day, slot = entry
        instructor_days[instructor].add(day)  # Add the day the instructor has a class
    
    # Step 2: Create a list to store the instructors with the number of days they have classes
    instructor_day_counts = []
    
    for instructor, days in instructor_days.items():
        num_days = len(days)  # Count the number of unique days
        instructor_day_counts.append((instructor, num_days, sorted(days)))  # Store the sorted days
    
    # Step 3: Sort the instructors based on the number of days (descending order)
    instructor_day_counts_sorted = sorted(instructor_day_counts, key=lambda x: x[1], reverse=True)
    
    # Step 4: Write the information to a file
    with open("instructors_sorted_by_days_on_campus"+date_time_str+".txt", "w") as f:
        f.write("Instructors sorted by the number of days they come to campus:\n\n")
        
        for instructor, num_days, days_list in instructor_day_counts_sorted:
            # Write the instructor, number of days, and the days they have a class
            f.write(f"Instructor: {instructor}, Number of Days: {num_days}, Days: {', '.join(days_list)}\n")
    
    print("Instructors sorted by number of days written to instructors_sorted_by_days_on_campus.txt")
    

# Second Part: After solving the model, print the sorted schedule


X_CS114_Kapleau, Jonathan_1_1_Friday_8:30-10:00 AM, 1.0

X_CS114_Kapleau, Jonathan_1_2_Friday_10:00-11:30 AM, 1.0

X_CS114_Zaidenberg, Ayelet_1_1_Tuesday_11:30-1:00 PM, 1.0

X_CS114_Zaidenberg, Ayelet_1_2_Thursday_11:30-1:00 PM, 1.0

X_CS114_Zaidenberg, Ayelet_2_1_Wednesday_8:30-10:00 AM, 1.0

X_CS114_Zaidenberg, Ayelet_2_2_Wednesday_10:00-11:30 AM, 1.0

X_CS116_Wu, Jun_1_1_Monday_11:30-1:00 PM, 1.0

X_CS116_Wu, Jun_1_2_Wednesday_11:30-1:00 PM, 1.0

X_CS116_Wu, Jun_2_1_Monday_10:00-11:30 AM, 1.0

X_CS116_Wu, Jun_2_2_Wednesday_10:00-11:30 AM, 1.0

X_CS241_Ionescu, Adrian_1_1_Monday_8:30-10:00 AM, 1.0

X_CS241_Ionescu, Adrian_1_2_Monday_10:00-11:30 AM, 1.0

X_CS241_Naik, Kamlesh_1_1_Tuesday_8:30-10:00 AM, 1.0

X_CS241_Naik, Kamlesh_1_2_Tuesday_10:00-11:30 AM, 1.0

X_CS241_Zaidenberg, Ayelet_1_1_Tuesday_10:00-11:30 AM, 1.0

X_CS241_Zaidenberg, Ayelet_1_2_Thursday_10:00-11:30 AM, 1.0

X_CS288_Dale, Mohit_1_1_Wednesday_1:00-2:30 PM, 1.0

X_CS288_Dale, Mohit_1_2_Friday_1:00-2:30 PM, 1.0

X_C

In [19]:
for v in instructor_soft_violated:
    if(variables[v].x>0.5):
        print(v,variables[v].x)


X_CS631_Theodoratos, Dimitrios_2_1_Tuesday_8:30-10:00 AM 1.0
X_CS631_Theodoratos, Dimitrios_1_1_Wednesday_8:30-10:00 AM 1.0
X_IT310_Patel, Dipesh_1_1_Thursday_8:30-10:00 AM 1.0


In [20]:






                
# Define valid start times for graduate courses
# valid_start_times = ["8:30-10:00 AM", "11:30-1:00 PM", "2:30-4:00 PM", "6:00-7:30 PM"]

# for course in df['Course'].unique():
#     for instructor in df['Instructor'].unique():
#         course_instructor_rows = df[(df['Course'] == course) & (df['Instructor'] == instructor)]
#         if course_instructor_rows.empty:
#             continue

#         course_number = int(course_instructor_rows['Course_Number'].iloc[0])

#         for section_id in range(1, course_instructor_rows.iloc[0]['# Sections'] + 1):
#             for slot in time_slots:

#                 # Constraints for courses with number < 600 (undergraduate)
#                 if course_number < 600:
#                     var_part1_monday = f"X_{course}_{instructor}_{section_id}_1_Monday_{slot}"
#                     var_part1_tuesday = f"X_{course}_{instructor}_{section_id}_1_Tuesday_{slot}"
#                     var_part1_wednesday = f"X_{course}_{instructor}_{section_id}_1_Wednesday_{slot}"
#                     var_part1_thursday = f"X_{course}_{instructor}_{section_id}_1_Thursday_{slot}"
#                     var_part1_friday = f"X_{course}_{instructor}_{section_id}_1_Friday_{slot}"

#                     # Part 1 on Monday: Part 2 on Wednesday or Thursday, same time slot
#                     if var_part1_monday in variables:
#                         var_part2_wednesday = f"X_{course}_{instructor}_{section_id}_2_Wednesday_{slot}"
#                         var_part2_thursday = f"X_{course}_{instructor}_{section_id}_2_Thursday_{slot}"
#                         model.addConstr(
#                             variables[var_part1_monday] <= variables[var_part2_wednesday] + variables[var_part2_thursday],
#                             name=f"part1_monday_part2_wed_thurs_{course}_{instructor}_{section_id}_{slot}"
#                         )

#                     # Part 1 on Tuesday: Part 2 on Thursday or Friday, same time slot
#                     if var_part1_tuesday in variables:
#                         var_part2_thursday = f"X_{course}_{instructor}_{section_id}_2_Thursday_{slot}"
#                         var_part2_friday = f"X_{course}_{instructor}_{section_id}_2_Friday_{slot}"
#                         model.addConstr(
#                             variables[var_part1_tuesday] <= variables[var_part2_thursday] + variables[var_part2_friday],
#                             name=f"part1_tuesday_part2_thurs_fri_{course}_{instructor}_{section_id}_{slot}"
#                         )

#                     # Part 1 on Wednesday: Part 2 on Friday, same time slot
#                     if var_part1_wednesday in variables:
#                         var_part2_friday = f"X_{course}_{instructor}_{section_id}_2_Friday_{slot}"
#                         model.addConstr(
#                             variables[var_part1_wednesday] <= variables[var_part2_friday],
#                             name=f"part1_wednesday_part2_friday_{course}_{instructor}_{section_id}_{slot}"
#                         )

#                     # Prevent scheduling Part 1 on Thursday or Friday unless both parts are on the same day
#                     if var_part1_thursday in variables:
#                         model.addConstr(variables[var_part1_thursday] == 0, name=f"no_part1_thursday_{course}_{instructor}_{section_id}_{slot}")
#                     if var_part1_friday in variables:
#                         model.addConstr(variables[var_part1_friday] == 0, name=f"no_part1_friday_{course}_{instructor}_{section_id}_{slot}")

                

#                    # For courses with number >= 600, schedule parts on consecutive time slots,
# # with the first time slot starting at one of the valid start times.
#                 if course_number >= 600:
#                     consecutive_schedule_vars = []  # Track consecutive scheduling variables for the section
                    
#                     for day in days:
#                         for idx, slot in enumerate(time_slots[:-1]):  # Exclude last slot for consecutive check
#                             var_part1 = f"X_{course}_{instructor}_{section_id}_1_{day}_{slot}"
#                             var_part2 = f"X_{course}_{instructor}_{section_id}_2_{day}_{time_slots[idx + 1]}"
                
#                             # Only apply this constraint if the first time slot is one of the valid start times
#                             if slot in valid_start_times and var_part1 in variables and var_part2 in variables:
#                                 # Create a binary variable to indicate whether this pair of time slots is used
#                                 var_consecutive = model.addVar(vtype=GRB.BINARY, name=f"Consecutive_{course}_{instructor}_{section_id}_{day}_{slot}")
#                                 consecutive_schedule_vars.append(var_consecutive)
                
#                                 # Ensure that if this consecutive pair is selected, both parts are scheduled
#                                 model.addConstr(
#                                     variables[var_part1] == var_consecutive,
#                                     name=f"link_part1_{course}_{instructor}_{section_id}_{day}_{slot}"
#                                 )
#                                 model.addConstr(
#                                     variables[var_part2] == var_consecutive,
#                                     name=f"link_part2_{course}_{instructor}_{section_id}_{day}_{slot}"
#                                 )
                
#                     # Ensure that exactly one valid consecutive pair is chosen for the section
#                     model.addConstr(
#                         sum(consecutive_schedule_vars) == 1,  # Only one consecutive pair should be selected
#                         name=f"one_consecutive_pair_{course}_{instructor}_{section_id}"
#                     )

# Define the specific time slot and day for the constraint

# This dictionary maps time slot abbreviations to their respective slot index


                        #    if email == 'usman@njit.edu':
                         #       usman_constraints.append(f"Day: {day_full}, Time Slot: {time_slot_full}")
#rint(" for usman:")
# if usman_constraints:
#     print("Constraints for usman@njit.edu:")
#     for constraint in usman_constraints:
#         print(constraint)






# Compute the sum of the slack variables

#print("section_capacity_map after calculation",section_capacity_map)




